# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saipan,15.1355,145.7010,28.39,61,75,9.77,MP,1708310674
1,1,margaret river,-33.9500,115.0667,27.36,42,100,4.00,AU,1708310675
2,2,lionel town,17.8101,-77.2406,27.94,84,46,3.12,JM,1708310675
3,3,keflavik,64.0049,-22.5624,4.24,93,75,3.09,IS,1708310676
4,4,stanley,54.8680,-1.6985,9.04,85,20,3.09,GB,1708310537


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    color = 'City'
    
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_condition_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)
                                              & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_weather_condition_df = ideal_weather_condition_df.dropna()

# Display sample data
ideal_weather_condition_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
84,84,cabo san lucas,22.8909,-109.9124,24.17,35,0,2.06,MX,1708310434
393,393,ciudad lazaro cardenas,17.9583,-102.2000,23.10,78,0,1.16,MX,1708310913
452,452,copala,16.6500,-98.9833,23.51,71,0,1.35,MX,1708310949
538,538,upington,-28.4478,21.2561,21.21,14,0,1.54,ZA,1708310780
543,543,la mira,18.0350,-102.3266,22.42,80,0,1.29,MX,1708310997


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_condition_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
84,cabo san lucas,MX,22.8909,-109.9124,35,
393,ciudad lazaro cardenas,MX,17.9583,-102.2000,78,
452,copala,MX,16.6500,-98.9833,71,
538,upington,ZA,-28.4478,21.2561,14,
543,la mira,MX,18.0350,-102.3266,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    "apiKey":geoapify_key      
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
copala - nearest hotel: No hotel found
upington - nearest hotel: Die Eiland Holiday Ressort
la mira - nearest hotel: Hotel La Loma


,City,Country,Lat,Lng,Humidity,Hotel Name
84,cabo san lucas,MX,22.8909,-109.9124,35,Comfort Rooms
393,ciudad lazaro cardenas,MX,17.9583,-102.2000,78,Hotel Sol del Pacífico
452,copala,MX,16.6500,-98.9833,71,No hotel found
538,upington,ZA,-28.4478,21.2561,14,Die Eiland Holiday Ressort
543,la mira,MX,18.0350,-102.3266,80,Hotel La Loma


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display
# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
   
    color = 'City',
    size = 'Humidity',
    hover_cols =['Hotel Name','Country']
)

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)